In [ ]:
#@title GPU Specification Check
!nvidia-smi

Wed Jul  2 13:17:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
#@title 1. Download dataset tinyshakespeare
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-07-02 13:17:13--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.006s  

2025-07-02 13:17:14 (171 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
#@title 2. Install dependencies (only needed in Colab)
!pip install transformers datasets accelerate streamlit pyngrok -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 130.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 123.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#@title 3. Load GPT-2 base model and tokenizer, test prompt
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Auto detect device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
tokenizer.pad_token = tokenizer.eos_token  # Add pad token if missing

# Test generation
input_prompt = "To be, or not to be"
input_ids = tokenizer(input_prompt, return_tensors="pt").input_ids.to(device)
output = model.generate(input_ids, max_new_tokens=50)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


To be, or not to be, a member of the Church of Jesus Christ of Latter-day Saints, I am not a member of the Church of Jesus Christ of Latter-day Saints. I am not a member of the Church of Jesus Christ of Latter-day Saints. I


In [ ]:
#@title 4. Prepare dataset for fine-tuning
from datasets import Dataset

# Load raw text into one long string
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Tokenize the entire text
tokens = tokenizer(text, return_special_tokens_mask=True, return_tensors="pt")["input_ids"][0]

# Split tokens into blocks of fixed length
block_size = 128
chunks = [tokens[i:i + block_size] for i in range(0, len(tokens) - block_size + 1, block_size)]

# Prepare dataset in HuggingFace format
input_ids = [chunk.tolist() for chunk in chunks]
attention_mask = [[1] * block_size] * len(input_ids)

# Convert to Dataset object
dataset = Dataset.from_dict({
    "input_ids": input_ids,
    "attention_mask": attention_mask
})

Token indices sequence length is longer than the specified maximum sequence length for this model (338025 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
#@title 5. Fine-tune GPT-2 with TinyShakespeare dataset
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned-shakespeare",
    overwrite_output_dir=True,
    per_device_train_batch_size=1,         # from: batch_size=1
    num_train_epochs=1,                    # ← keep 1 epoch (you can set steps below)
    max_steps=3000,                        # from: num_steps=3000
    logging_steps=100,
    save_steps=1000,                       # from: save_every=1000
    save_total_limit=1,
    learning_rate=1e-3,                    # from: learning_rate=1e-3
    fp16=False,                            # from: fp16=False
    prediction_loss_only=True,
    push_to_hub=False,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start training
trainer.train()

# Save fine-tuned model
model.save_pretrained("./gpt2-finetuned-shakespeare")
tokenizer.save_pretrained("./gpt2-finetuned-shakespeare")

/tmp/ipython-input-6-2524703578.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: herutriana44 (gen-apalah) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,5.087900
200,4.909300
300,4.711500
400,4.551300
500,4.540500
600,4.339500
700,4.278400
800,4.188700
900,4.204000
1000,4.142000


('./gpt2-finetuned-shakespeare/tokenizer_config.json',
 './gpt2-finetuned-shakespeare/special_tokens_map.json',
 './gpt2-finetuned-shakespeare/vocab.json',
 './gpt2-finetuned-shakespeare/merges.txt',
 './gpt2-finetuned-shakespeare/added_tokens.json')

In [ ]:
#@title 6. Load fine-tuned model and generate text
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load saved model
model = GPT2LMHeadModel.from_pretrained("./gpt2-finetuned-shakespeare").to(device)
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-finetuned-shakespeare")

In [ ]:
#@title 7. Try a new prompt
prompt = "ROMEO:"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
output = model.generate(inputs["input_ids"], max_new_tokens=100)

print(tokenizer.decode(output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ROMEO:
I thank you, sir, I thank you.

ROMEO:
I thank you, sir, for your honour.

JULIET:
I thank you, sir, for your honour.

ROMEO:
I thank you, sir, for your honour.

JULIET:
I thank you, sir, for your honour.

ROMEO:
I thank you, sir, for your honour.

JULI


In [ ]:
#@title 8. Write streamlit & ngrok program to implement GPT2 Fine-tuned model
%%writefile app.py
import streamlit as st
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load fine-tuned GPT-2 model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPT2LMHeadModel.from_pretrained("./gpt2-finetuned-shakespeare").to(device)
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-finetuned-shakespeare")

# Set padding token if not already set
tokenizer.pad_token = tokenizer.eos_token

# Streamlit Interface
st.set_page_config(page_title="GPT-2 Text Generator", layout="centered")
st.title("🧠 GPT-2 Fine-tuned Text Generator")

st.markdown("This app uses a GPT-2 model fine-tuned on your dataset to generate free-form English text.")

prompt = st.text_area("Enter your prompt below:", value="Once upon a time", height=100)

max_new_tokens = st.slider("Max tokens to generate", min_value=20, max_value=300, value=100)

temperature = st.slider("Sampling temperature", min_value=0.1, max_value=1.5, value=1.0)

top_k = st.slider("Top-k sampling", min_value=0, max_value=100, value=50)

do_sample = st.checkbox("Use sampling?", value=True)

if st.button("Generate"):
    with st.spinner("Generating text..."):
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
        output = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k if top_k > 0 else None,
            do_sample=do_sample,
            pad_token_id=tokenizer.eos_token_id
        )
        result = tokenizer.decode(output[0], skip_special_tokens=True)
        st.markdown("### Generated Output:")
        st.write(result)


Writing app.py


In [ ]:
#@title Run streamlit and ngrok code using app.py
from pyngrok import ngrok

# Kill any existing tunnels
ngrok.kill()

NGROK_AUTH_TOKEN = "YOUR_NGROK_AUTHTOKEN" #@param {type:"string"}
!ngrok authtoken $NGROK_AUTH_TOKEN

# Open ngrok tunnel for port 8501 (Streamlit default)
public_url = ngrok.connect(8501)
print(f"🚀 Public URL: {public_url}")

# Start Streamlit app
!streamlit run app.py &>/content/log.txt &

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🚀 Public URL: NgrokTunnel: "https://d37b-34-143-172-14.ngrok-free.app" -> "http://localhost:8501"
